<a href="https://www.kaggle.com/code/sonujha090/image-classification-indoor-scence?scriptVersionId=110412195" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# importing necessary libraries
import os 
from pathlib import Path
from glob import glob
import random

# Path to the dataset
path = Path('../input/indoor-scenes-cvpr-2019')
image_path = path/'indoorCVPR_09/Images' # Path to the images
image_files = glob(str(image_path/'**/*.jpg')) # List of all the images

# shuffle images
random.shuffle(image_files)

# label encoding for the classes
class_names = [p.name for p in image_path.glob('*') if p.is_dir()]

label2index = {v:k for k,v in enumerate(class_names)}
index2label = {v:k for k,v in label2index.items()}

# Creating a dataframe with the image path and the corresponding label
import pandas as pd
df = pd.DataFrame({'image':image_files})
df['label'] = df['image'].map(lambda x: label2index[x.split('/')[-2]])

# Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Saving the train and test dataframe
train_df.to_csv('/train.csv', index=False)
test_df.to_csv('/test.csv', index=False)


In [2]:
from torchvision import transforms
import torch 

class config:
    EPOCHS = 5
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    LOAD_MODEL = True
    SAVE_MODEL = True
    CHECKPOINT_FILE = "my_checkpoint.pth.tar"

    tfms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # save checkpoint
    def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
        print("=> Saving checkpoint")
        torch.save(state, filename)

    # load checkpoint
    def load_checkpoint(checkpoint, model):
        print("=> Loading checkpoint")
        model.load_state_dict(checkpoint['state_dict'])

    # Learning rate scheduler
    def lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=5):
        """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
        lr = init_lr * (0.1**(epoch // lr_decay_epoch))
        if epoch % lr_decay_epoch == 0:
            print('LR is set to {}'.format(lr))

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        return optimizer  

In [3]:
from torch.utils.data import Dataset, DataLoader, Subset 
import pandas as pd 
import torch, torchvision
from PIL import Image 
from torchvision import transforms 

class IndoorSceneDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['image']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, row['label']

if __name__=='__main__':
    df = pd.read_csv('/train.csv')
    ds = IndoorSceneDataset(df, transform=config.tfms)
    dl = DataLoader(ds, batch_size=32, shuffle=True)
    for xb, yb in dl:
        print(xb.shape, yb.shape)
        break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [4]:
from tqdm import tqdm
import numpy as np 
import torch
from sklearn.metrics import accuracy_score

def train_model(model, train_dl, valid_dl, optimizer, criterion, num_epochs=10, device=config.device):
    train_losses, valid_losses, train_acc, valid_acc = [], [], [], []
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        for x, y in tqdm(train_dl):
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            accuracy = accuracy_score(y.detach().cpu().numpy(), y_pred.argmax(dim=1).detach().cpu().numpy())
            train_acc.append(accuracy)
        model.eval()
        for x, y in tqdm(valid_dl):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            accuracy = accuracy_score(y.detach().cpu().numpy(), y_pred.argmax(dim=1).detach().cpu().numpy())
            valid_losses.append(loss.item())
            valid_acc.append(accuracy)
        print(f'Epoch: {epoch+1}, Train Loss: {np.mean(train_losses):.4f}, Train Accuracy: {np.mean(train_acc):.4f}, Valid Loss: {np.mean(valid_losses):.4f}, Valid Accuracy: {np.mean(valid_acc):.4f}')

In [5]:
%%time 

import torch, torchvision
from torch.utils.data import Dataset, DataLoader, Subset 
# from dataset import IndoorSceneDataset, tfms
import pandas as pd
import sys 


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_df = pd.read_csv('/train.csv')
valid_df = pd.read_csv('/test.csv')

train_dataset = IndoorSceneDataset(train_df, transform=config.tfms)
valid_dataset = IndoorSceneDataset(valid_df, transform=config.tfms)

# train_dataset = Subset(train_dataset, range(0, int(len(train_dataset)*0.1)))
# valid_dataset = Subset(train_dataset, range(0, int(len(valid_dataset)*0.1)))

train_dl = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=config.BATCH_SIZE, shuffle=True)


model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Linear(512, 67)

# load checkpoint
if config.LOAD_MODEL and config.CHECKPOINT_FILE in os.listdir():
    load_checkpoint(torch.load(config.CHECKPOINT_FILE), model)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model = model.to(device) 

train_model(model, train_dl, valid_dl, optimizer, criterion, num_epochs=config.EPOCHS)

# save checkpoint
if config.SAVE_MODEL:
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    config.save_checkpoint(checkpoint)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

100%|██████████| 98/98 [00:58<00:00,  1.67it/s]


Epoch: 1, Train Loss: 2.5275, Train Accuracy: 0.3978, Valid Loss: 1.7610, Valid Accuracy: 0.5421


100%|██████████| 98/98 [00:37<00:00,  2.60it/s]


...

Epoch: 3, Train Loss: 1.7913, Train Accuracy: 0.5441, Valid Loss: 1.5352, Valid Accuracy: 0.5901


100%|██████████| 98/98 [00:36<00:00,  2.67it/s]


Epoch: 4, Train Loss: 1.6362, Train Accuracy: 0.5762, Valid Loss: 1.4746, Valid Accuracy: 0.6034


100%|██████████| 98/98 [00:37<00:00,  2.61it/s]

Epoch: 5, Train Loss: 1.5240, Train Accuracy: 0.6004, Valid Loss: 1.4380, Valid Accuracy: 0.6110
=> Saving checkpoint
CPU times: user 14min 30s, sys: 13 s, total: 14min 43s
Wall time: 17min 23s


In [6]:
# load checkpoint
if config.LOAD_MODEL and config.CHECKPOINT_FILE in os.listdir():
    config.load_checkpoint(torch.load(config.CHECKPOINT_FILE), model)

model = model.to(device) 

train_model(model, train_dl, valid_dl, optimizer, criterion, config.EPOCHS)

# save checkpoint
if config.SAVE_MODEL:
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    config.save_checkpoint(checkpoint)

=> Loading checkpoint


100%|██████████| 98/98 [00:37<00:00,  2.64it/s]


Epoch: 1, Train Loss: 1.0103, Train Accuracy: 0.7094, Valid Loss: 1.2598, Valid Accuracy: 0.6430


100%|██████████| 98/98 [00:37<00:00,  2.59it/s]


Epoch: 2, Train Loss: 0.9853, Train Accuracy: 0.7161, Valid Loss: 1.2720, Valid Accuracy: 0.6415


...

Epoch: 3, Train Loss: 0.9664, Train Accuracy: 0.7194, Valid Loss: 1.2783, Valid Accuracy: 0.6382


100%|██████████| 98/98 [00:38<00:00,  2.54it/s]


Epoch: 4, Train Loss: 0.9491, Train Accuracy: 0.7250, Valid Loss: 1.2830, Valid Accuracy: 0.6385


100%|██████████| 98/98 [00:39<00:00,  2.51it/s]

Epoch: 5, Train Loss: 0.9295, Train Accuracy: 0.7297, Valid Loss: 1.2832, Valid Accuracy: 0.6399
=> Saving checkpoint
